In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from math import floor
from math import isinf
from os import path
from sys import path as syspath

syspath.append("../../perps")
import modules.figures as figs
import modules.perps as perps

plots_path = "../../../papers/perps-overview/plots/"

def plot(S, F, r, a, b, plots_path=None, show=False):
    x_axis = (F - S) / S
    R = np.array(
        [
            (f - s) / s + min(a, max(b, ((1 + r) * s - f) / s))
            for f, s in np.column_stack((F, S))
        ]
    )

    plt.ioff()
    plt.clf()

    plt.plot(x_axis, R)
    plt.xlabel(r"$\frac{F-S}{S}$")
    plt.ylabel(r"$R$")
    plt.xticks(np.arange(min(x_axis), 1.1 * max(x_axis), abs(min(x_axis))))
    if isinf(a) and isinf(-b):
        plt.title("r = {r:.2%}, a = {a}, b = {b}".format(r=r, a=a, b=b))
        plt.yticks(np.arange(0.5 * min(R), 1.5 * max(R), 0.5 * abs(min(R))))
    else:
        plt.title("r = {r:.2%}, a = {a:.2%}, b = {b:.2%}".format(r=r, a=a, b=b))
    if not plots_path is None:
        plt.savefig(
            path.join(plots_path, "r_{r}_a_{a}_b_{b}.png".format(r=r, a=a, b=b))
        )
    if show:
        plt.show()

def timedelta_formatter(delta: dt.timedelta) -> str:
    s = ""
    minutes = delta.seconds/60
    hours = floor(minutes/60)
    minutes = floor(minutes-hours*60)
    if delta.days > 0:
        s += f"{delta.days}d "
    if hours > 0:
        s += f"{hours}h "
    if minutes > 0:
        s += f"{minutes}min "
    if delta.seconds%60 > 0:
        if s != "":
            s += " "
        s += f"{delta.seconds%60}s "
    return s[:-1]

def compute_table(start_date, end_date, spot_curve, perp_curve, funding_frequencies, spot_sampling_frequencies, perp_sampling_frequencies):
    df = pd.DataFrame(columns=["funding_frequency", "perp_funding_frequency", "spot_funding_frequency", "spot_rr", "perp_rr", "sum_funding_payments"])
    for ff in funding_frequencies:
        funding_schedule = perps.create_schedule(
            start_date,
            end_date,
            ff,
        )
        for sf in spot_sampling_frequencies:
            spot_sampling_schedule = perps.create_schedule(
                start_date, end_date, sf
            )
            for pf in perp_sampling_frequencies:
                perp_sampling_schedule = perps.create_schedule(
                    start_date, end_date, pf
                )
                funding_periods = perps.compute_funding_periods(
                    funding_schedule,
                    spot_sampling_schedule,
                    perp_sampling_schedule,
                    spot_curve,
                    perp_curve,
                )
                spot_rr, perp_rr, sum_funding_payments = perps.summarise(
                    start_date, end_date,
                    funding_periods, spot_curve, perp_curve)
                df.loc[len(df.index)] = [ff, pf, sf, spot_rr, perp_rr, sum_funding_payments]
    return df

In [ ]:
# funding rate plots
r = 0.0001
grid = np.linspace(-0.001, 0.001, 201)
S_const = 100
S = np.array([S_const for x in grid])
F = np.array([S_const * (1 + x) for x in grid])

plot(S, F, r, float("inf"), -float("inf"), plots_path=plots_path);
plot(S, F, r, 0.0005, -0.0005, plots_path=plots_path);
plot(S, F, r, 0, 0, plots_path=plots_path);

In [ ]:
start_date = dt.datetime(2023, 5, 1)
end_date = dt.datetime(2023, 6, 1)
spot_curve_1_path = "../../perps/data/spot_1.csv"
perp_curve_1_path = "../../perps/data/perps_1.csv"

spot_curve, perp_curve = figs.spot_perp_difference(
    start_date=start_date,
    end_date=end_date,
    use_spot_curve_1=True,
    spot_curve_1_path=spot_curve_1_path,
    use_perp_curve_1=True,
    perp_curve_1_path=perp_curve_1_path,
    show=False,
    plot_save_path=path.join(plots_path, "example_curves.png"),
);

In [ ]:
funding_schedule_days = 7
spot_sampling_hours = 8
perp_sampling_seconds = 300

baseline_funding_payment_frequency = dt.timedelta(days=funding_schedule_days)
baseline_spot_sampling_frequency = dt.timedelta(hours=spot_sampling_hours)
baseline_perp_sampling_frequency = dt.timedelta(seconds=perp_sampling_seconds)

figs.funding_periods(
    start_date=start_date,
    end_date=end_date,
    spot_curve=spot_curve,
    perp_curve=perp_curve,
    funding_payment_frequency=baseline_funding_payment_frequency,
    spot_sampling_frequency=baseline_spot_sampling_frequency,
    perp_sampling_frequency=baseline_perp_sampling_frequency,
    show=False,
    show_funding_payment=True,
    show_funding_rate=False,
    show_spot_twap=True,
    show_perp_twap=True,
    show_spot_step=True,
    show_perp_step=True,
    show_spot_points=False,
    show_perp_points=False,
    plot_save_path=path.join(
        plots_path,
        f"payment-frequency-{funding_schedule_days}d_spot-sampling-{spot_sampling_hours}h_perp-sampling-{perp_sampling_seconds}s.png",
    ),
);

In [ ]:
funding_frequencies = [dt.timedelta(days=7),dt.timedelta(days=1),dt.timedelta(hours=12),dt.timedelta(hours=6),dt.timedelta(hours=3),dt.timedelta(hours=1),dt.timedelta(minutes=30),dt.timedelta(minutes=10),dt.timedelta(minutes=5)]
df_vary_funding_frequencies = compute_table(start_date=start_date, end_date=end_date, spot_curve=spot_curve, perp_curve=perp_curve, funding_frequencies=funding_frequencies, spot_sampling_frequencies=[baseline_spot_sampling_frequency], perp_sampling_frequencies=[baseline_perp_sampling_frequency])
with open(path.join(plots_path, "table_vary_funding_frequency.tex"), "w") as file:
    latex_table =  df_vary_funding_frequencies.drop(columns=["perp_funding_frequency", "spot_funding_frequency"]).to_latex(
                formatters={"funding_frequency": timedelta_formatter},
                float_format="{:.2E}".format)
    file.write(latex_table)

In [ ]:
spot_sampling_frequencies = [dt.timedelta(days=7),dt.timedelta(days=1),dt.timedelta(hours=12),dt.timedelta(hours=6),dt.timedelta(hours=3),dt.timedelta(hours=1),dt.timedelta(minutes=30),dt.timedelta(minutes=10),dt.timedelta(minutes=5)]

df_vary_spot_sampling  = compute_table(start_date=start_date, end_date=end_date, spot_curve=spot_curve, perp_curve=perp_curve, funding_frequencies=[baseline_funding_payment_frequency], spot_sampling_frequencies=spot_sampling_frequencies, perp_sampling_frequencies=[baseline_perp_sampling_frequency])
with open(path.join(plots_path, "table_vary_spot_sampling_frequency.tex"), "w") as file:
    latex_table =  df_vary_spot_sampling.drop(columns=["perp_funding_frequency", "funding_frequency"]).to_latex(
                formatters={"spot_funding_frequency": timedelta_formatter},
                float_format="{:.2E}".format)
    file.write(latex_table)